## Planificación de la producción: sector industrial

V1.0, Jul 2023

#### Problema CQM c/ reformulación Qubo

 Una fábrica de electrodomésticos dispone de dos cadenas de montaje. 
 
- La productividad por hora de ambas cadenas es:

    - la cadena A produce 10 lavadoras y 5 frigoríficos

    - la cadena B produce 7 lavadoras y 6 frigoríficos. 


- El coste por hora de cada cadena es:

    - cadena A: 1200€
    - cadena B: 1500€

- La cadena A es a doble turno, pero no debe sobrepasar el doble de horas que la cadena B

- No se deben de sobrepasar las 24 horas diarias entre ambas líneas

- Se quieren producir como mínimo 200 lavadoras y 120 frigoríficos diarios


<b>Obtener las horas de funcionamiento de ambas cadenas con el objetivo de minimizar costes</b>


<b>Planteamiento algebraico:</b>

    - x1, x2 ; variables cadenas A y B

    - Función objetivo: minimizar{f(x1,x2)} = -1200*x1 - 1500*x2

- Restricciones:

    - 10x1 + 7x2 >= 200
    - 5x1 + 6x2 >= 140
    - x1 <= 2X2
    - x1 + x2 <= 24
 


In [61]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset,n=None):
    # Resultados
    
    subset=sampleset.filter(lambda s: s.is_feasible).aggregate()
    
    if not n:
        print(f"\nLas {len(subset)} soluciones factibles al problema son:\n")
    else:
        print(f"\nImprimiendo las {n}/{len(subset)} primeras soluciones factibles:\n")
    
    print(subset.slice(n))
    
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)

In [76]:
# Definición del problema CQM

x1 = Integer('Xa',upper_bound=20)
x2 = Integer('Xb',upper_bound=20)

cqm = ConstrainedQuadraticModel()


# Función objetivo: minimizar coste

cqm.set_objective(12*x1 + 15*x2)

# Restricciones

cqm.add_constraint(10*x1 + 7*x2 >= 200,"Producción lavadoras: 10Xa + 7Xb >=200")
cqm.add_constraint(5*x1 + 6*x2 >= 120,"Producción frigos: 5Xa + 6Xa >= 120")
cqm.add_constraint(x1 - 2*x2 <= 0,"Horas: x1 <= 2x2")
cqm.add_constraint(x1 + x2 <= 24,"Horas diarias: x1 + x2 <= 24")

print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 4 constraints, 10 biases

Objective
  -12*Integer('Xa') - 15*Integer('Xb')

Constraints
  Producción lavadoras: 10Xa + 7Xb >=200: 10*Integer('Xa') + 7*Integer('Xb') >= 200.0
  Producción frigos: 5Xa + 6Xa >= 120: 5*Integer('Xa') + 6*Integer('Xb') >= 120.0
  Horas: x1 <= 2x2: Integer('Xa') - 2*Integer('Xb') <= 0.0
  Horas diarias: x1 + x2 <= 24: Integer('Xa') + Integer('Xb') <= 24.0

Bounds
  0.0 <= Integer('Xa') <= 20.0
  0.0 <= Integer('Xb') <= 20.0



### Solución por fuerza bruta

In [77]:
result = ExactCQMSolver().sample_cqm(cqm)

In [78]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/9 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 11 13 -327.0       1 arra...    True
1 12 12 -324.0       1 arra...    True
2 13 11 -321.0       1 arra...    True
3 14 10 -318.0       1 arra...    True
4 15  9 -315.0       1 arra...    True
5 16  8 -312.0       1 arra...    True
6 13 10 -306.0       1 arra...    True
7 14  9 -303.0       1 arra...    True
8 15  8 -300.0       1 arra...    True
['INTEGER', 9 rows, 9 samples, 2 variables]



<b>Resultados:</b>

- El coste mínimo logrando objetivos es de 30.000€

- La planta A debe permanecer activa 15 horas

- La planta B trabaja 8 horas

### Solución con QPU

In [74]:
#reformulación qubo y sampling

fl=20 # factor penalizador de lagrange

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=3000)

In [75]:
sampleset=sol_problema(result)
solQPU=sol_factibles(sampleset,5)


Imprimiendo las 5/4 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 15  8  300.0       2 arra...    True
1 16  8  312.0       1 arra...    True
2 15  9  315.0       1 arra...    True
3 13 11  321.0       1 arra...    True
['INTEGER', 4 rows, 5 samples, 2 variables]




<b>Analizando los resultados del sampler QPU</b>

- Se ha formulado un problema Qubo desde un problema CQM

- El espacio de soluciones factibles es similar al de fuerza bruta (sobre 350)

- Se han replicado los valores óptimos obtenidos por fuerza bruta

- Se necesitaron entre 2000-3000 samples, y después de ajustar la función objetivo a 12*x1 + 15*x2 ya que en caso contrario no convergía con reads sensatos.
